In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


# Column 제거

In [3]:
train_df = pd.read_csv('../data/train.csv')
train_df.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,0,1,106,지방도1112호선,0,0,60.0,0.0,32400.0,0.0,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,0,2,103,일반국도11호선,0,0,60.0,0.0,0.0,0.0,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,0,2,103,일반국도16호선,0,0,80.0,0.0,0.0,0.0,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,0,2,107,태평로,0,0,50.0,0.0,0.0,0.0,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,0,2,103,일반국도12호선,0,0,80.0,0.0,0.0,0.0,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [4]:
train_df.columns

Index(['id', 'base_date', 'day_of_week', 'base_hour', 'road_in_use',
       'lane_count', 'road_rating', 'road_name', 'multi_linked',
       'connect_code', 'maximum_speed_limit', 'vehicle_restricted',
       'weight_restricted', 'height_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target'],
      dtype='object')

In [5]:
test_df = pd.read_csv('../data/test.csv')
test_df.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,목,17,0,3,107,연삼로,0,0,70.0,0.0,0.0,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음
1,TEST_000001,20220809,화,12,0,2,103,일반국도12호선,0,0,70.0,0.0,0.0,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음
2,TEST_000002,20220805,금,2,0,1,103,일반국도16호선,0,0,60.0,0.0,0.0,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음
3,TEST_000003,20220818,목,23,0,3,103,일반국도11호선,0,0,70.0,0.0,0.0,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음
4,TEST_000004,20220810,수,17,0,3,106,번영로,0,0,70.0,0.0,0.0,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음


In [6]:
drop_col = ['id', 'vehicle_restricted', 'height_restricted', 'start_latitude', 'start_longitude', #'base_date'
            'end_latitude', 'end_node_name', 'end_longitude', 'road_in_use']
x = train_df.drop(drop_col, axis = 1)
x = x.drop('target', axis = 1)
y = train_df.loc[:, 'target']

test_data = test_df.drop(drop_col, axis = 1)

In [7]:
# import statsmodels.api as sm
# for i in train_df.columns:
#     model = sm.Logit(train_df[i], train_df['target'])
#     result = model.fit()
#     print(result.pvalues)

In [8]:
dumm_cols = ['day_of_week', 'base_hour', 'road_rating', 'road_name', 
             'multi_linked', 'connect_code', 'road_type', 
             'start_turn_restricted', 'end_turn_restricted', 'start_node_name']

In [9]:
for col in dumm_cols:
    ohe = OneHotEncoder(sparse=False) # 나중에 drop 적용해보기
    
    temp_x = ohe.fit_transform(x[col].values.reshape(-1,1))
#     temp_x_df = pd.DataFrame(temp_x, columns=ohe.categories_[0])
    temp_x_df = pd.DataFrame(temp_x, columns=ohe.get_feature_names([col]))
    x.drop(col, axis=1, inplace = True)
    x = pd.concat([x, temp_x_df], axis = 1)
    
    temp_test = ohe.transform(test_data[col].values.reshape(-1,1))
#     temp_test_df = pd.DataFrame(temp_test, columns=ohe.categories_[0])
    temp_test_df = pd.DataFrame(temp_test, columns=ohe.get_feature_names([col]))
    test_data.drop(col, axis=1, inplace = True)
    test_data = pd.concat([test_data, temp_test_df], axis = 1)

In [10]:
# for col in dumm_cols:
#     le = LabelEncoder()
#     x[col] = le.fit_transform(x[col])
#     test_data[col] = le.transform(x_test[col])

In [11]:
x.shape

(4701217, 596)

In [12]:
test_data.shape

(291241, 596)

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 2022)

In [13]:
# x_train.to_csv('../data/train1_origin/x_train.csv', index=False)
# x_val.to_csv('../data/train1_origin/x_val.csv', index=False)
# y_train.to_csv('../data/train1_origin/y_train.csv', index=False)
# y_val.to_csv('../data/train1_origin/y_val.csv', index=False)
# test_data.to_csv('../data/train1_origin/x_test.csv', index=False)

# XGB

In [14]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(n_estimators=100,gamma=1,eta=0.1,max_depth=5,reg_lambda=5,reg_alpha=5,random_state=2022)

model_xgb.fit(x_train, y_train)



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=1, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=2022,
             reg_alpha=5, ...)

In [16]:
y_pred = model_xgb.predict(test_data)

In [18]:
result = pd.DataFrame({'id':test_df['id'], 'target':y_pred})
result.head()

,id,target
0,TEST_000000,28.699699
1,TEST_000001,45.003902
2,TEST_000002,55.764816
3,TEST_000003,31.750568
4,TEST_000004,43.394955


In [19]:
result.to_csv('../data/submission_v3.csv', index=False)

# Jae Kyeong's

In [21]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRFRegressor
from lightgbm import LGBMRegressor
# from lightgbm import DaskLGBMClassifier

In [24]:
rf_model = RandomForestRegressor(n_estimators=300,
              max_depth=30,
              max_leaf_nodes=32,
              random_state=42,
              n_jobs=-1
              )

xgb_model = XGBRFRegressor(max_depth=30,
                n_estimators=300,
                learning_rate=0.01,
                gamma=0, subsample=0.75,
                use_label_encoder=False,
                colsample_bytree=1,
                eval_metric='logloss',
                n_jobs=-1
                # subsample=0.7
                )

xgb_meta = XGBRFRegressor(max_depth=27,
                n_estimators=500,
                learning_rate=0.005,
                gamma=0, subsample=0.75,
                use_label_encoder=False,
                colsample_bytree=1,
                eval_metric='logloss',
                n_jobs=-1
                # subsample=0.7
                )

lgb_model = LGBMRegressor(
                max_depth= 30,
                n_estimators= 300,
                learning_rate= 0.001,
                num_leaves = 40,
                n_jobs=-1)

# Second Layer
lgb_meta = LGBMRegressor(
                max_depth= 25,
                n_estimators= 600,
                learning_rate= 0.001,
                num_leaves = 32,
                n_jobs=-1
                )

mlp_model = MLPRegressor(hidden_layer_sizes=(32,), random_state=1, max_iter=10, validation_fraction=0.15, warm_start=False)



In [25]:
vr = VotingRegressor([('lgb', lgb_meta), ('mlp', mlp_model)], n_jobs=-1)

In [ ]:
estimators = [('rf', rf_model), ('xgb', xgb_model), ('lgb_1', lgb_model)]

model = StackingRegressor(estimators=estimators,
               final_estimator=vr,
               verbose=1,
               n_jobs=-1)

model.fit(x_train, y_train).score(x_train, y_train) # score: R2 score 반환

# Pycaret

In [15]:
# from pycaret.regression import *

In [ ]:
# setup(session_id = 2022, data = pd.concat([x_train,y_train],axis=1), target = 'target',
#       test_data = pd.concat([x_val,y_val],axis=1),
#       normalize = True, normalize_method = 'zscore',
#       transformation=True, 
#       #fold_strategy='stratifiedkfold', 
#       use_gpu = True)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:48:23
Status,. . . . . . . . . . . . . . . . . .,Preprocessing Data


In [ ]:
# pycaret_regression_models = compare_models(n_select=15, sort='MAE')

In [ ]:
# models()